To use it you need the next files:
    - 'test.hdf'
    - 'train1.hdf'
    - 'train2.hdf'
    - 'test_private.hdf'
    - 'features.npy'
    - 'add_features.npy'

In [1]:
import os
import numpy as np
import pandas as pd
import h5py
import lightgbm as lgb
import scoring
import matplotlib.pyplot as plt
import tqdm

from hep_ml import splot
from sklearn.linear_model import LogisticRegression

Open files

In [3]:
train_hdf1 = h5py.File('train1.hdf', mode = 'r')
train_hdf2 = h5py.File('train2.hdf', mode = 'r')
features = np.load('features.npy')
add_features = np.load('add_features.npy')
print(add_features)

['sWeight' 'particle_type' 'label' 'kinWeight' 'weight']


Choose features of 0, 1, 2, 3 stations

In [4]:
features_0 = [0, 4, 8, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 54, 58, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
       81, 82, 83, 84, 85, 86, 87]
features_1 = [1,  5,  8, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 53, 55, 59, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
              75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
features_2 = [ 2,  6, 8, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 53, 56, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
              77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
features_3 = [ 3,  7, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 53, 57, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
              74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]

Create models for 0, 1, 2, 3 stations + to determine muon&pion and muon&proton

In [5]:
model0 = lgb.LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
                            class_weight=None, colsample_bytree=0.8686735616702099,
                            feature_fraction=0.8, importance_type='split', learning_rate=0.1,
                            max_depth=-1, metric='auc', min_child_samples=5,
                            min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
                            n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
                            reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
                            save_binary=True, silent=True, subsample=1.0,
                            subsample_for_bin=320000, subsample_freq=0)
model1 = lgb.LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
                            class_weight=None, colsample_bytree=0.8686735616702099,
                            feature_fraction=0.8, importance_type='split', learning_rate=0.1,
                            max_depth=-1, metric='auc', min_child_samples=5,
                            min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
                            n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
                            reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
                            save_binary=True, silent=True, subsample=1.0,
                            subsample_for_bin=320000, subsample_freq=0)
model2 = lgb.LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
                            class_weight=None, colsample_bytree=0.8686735616702099,
                            feature_fraction=0.8, importance_type='split', learning_rate=0.1,
                            max_depth=-1, metric='auc', min_child_samples=5,
                            min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
                            n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
                            reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
                            save_binary=True, silent=True, subsample=1.0,
                            subsample_for_bin=320000, subsample_freq=0)
model3 = lgb.LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
                            class_weight=None, colsample_bytree=0.8686735616702099,
                            feature_fraction=0.8, importance_type='split', learning_rate=0.1,
                            max_depth=-1, metric='auc', min_child_samples=5,
                            min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
                            n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
                            reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
                            save_binary=True, silent=True, subsample=1.0,
                            subsample_for_bin=320000, subsample_freq=0)
model_pion = lgb.LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
                            class_weight=None, colsample_bytree=0.8686735616702099,
                            feature_fraction=0.8, importance_type='split', learning_rate=0.1,
                            max_depth=-1, metric='auc', min_child_samples=5,
                            min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
                            n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
                            reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
                            save_binary=True, silent=True, subsample=1.0,
                            subsample_for_bin=320000, subsample_freq=0)
model_proton = lgb.LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
                            class_weight=None, colsample_bytree=0.8686735616702099,
                            feature_fraction=0.8, importance_type='split', learning_rate=0.1,
                            max_depth=-1, metric='auc', min_child_samples=5,
                            min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
                            n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
                            reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
                            save_binary=True, silent=True, subsample=1.0,
                            subsample_for_bin=320000, subsample_freq=0)

Using of absolut amount of weights cause the better score

In [6]:
abs_weights = np.abs(np.concatenate((np.array(train_hdf1['add_features'][:, 4]), np.array(train_hdf2['add_features'][:, 4]))))

Divide the labels and features for pion/muon and proton/muon

In [7]:
train_hdf_div = h5py.File('train_div.hdf', mode = 'r')
particle_type = np.array(train_hdf_div['particle_type'])
labels_pion = particle_type[(particle_type == 1) + (particle_type == 0)].copy()
labels_proton = particle_type[(particle_type == 1) + (particle_type == 2)].copy()
labels_proton[labels_proton == 2] = 0
train_hdf_div.close()

Training of all models

In [8]:
%time model_pion.fit(np.concatenate((np.array(train_hdf1['features']), np.array(train_hdf2['features'])))[(particle_type == 1) + (particle_type == 0)], labels_pion, sample_weight=abs_weights[(particle_type == 1) + (particle_type == 0)])

CPU times: user 1h 9min 12s, sys: 41.4 s, total: 1h 9min 53s
Wall time: 8min 55s


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8686735616702099,
        feature_fraction=0.8, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='auc', min_child_samples=5,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
        reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
        save_binary=True, silent=True, subsample=1.0,
        subsample_for_bin=320000, subsample_freq=0)

In [9]:
%time model_proton.fit(np.concatenate((np.array(train_hdf1['features']), np.array(train_hdf2['features'])))[(particle_type == 1) + (particle_type == 2)], labels_proton, sample_weight=abs_weights[(particle_type == 1) + (particle_type == 2)])

CPU times: user 1h 12min 51s, sys: 43.9 s, total: 1h 13min 35s
Wall time: 9min 22s


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8686735616702099,
        feature_fraction=0.8, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='auc', min_child_samples=5,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
        reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
        save_binary=True, silent=True, subsample=1.0,
        subsample_for_bin=320000, subsample_freq=0)

In [10]:
%time model0.fit(np.concatenate((np.array(train_hdf1['features'][:, features_0]), np.array(train_hdf2['features'][:, features_0]))), np.concatenate((np.array(train_hdf1['labels']), np.array(train_hdf2['labels']))), sample_weight=abs_weights)

CPU times: user 40min 37s, sys: 32.3 s, total: 41min 10s
Wall time: 5min 14s


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8686735616702099,
        feature_fraction=0.8, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='auc', min_child_samples=5,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
        reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
        save_binary=True, silent=True, subsample=1.0,
        subsample_for_bin=320000, subsample_freq=0)

In [11]:
%time model1.fit(np.concatenate((np.array(train_hdf1['features'][:, features_1]), np.array(train_hdf2['features'][:, features_1]))), np.concatenate((np.array(train_hdf1['labels']), np.array(train_hdf2['labels']))), sample_weight=abs_weights)

CPU times: user 40min 28s, sys: 30.9 s, total: 40min 59s
Wall time: 5min 13s


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8686735616702099,
        feature_fraction=0.8, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='auc', min_child_samples=5,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
        reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
        save_binary=True, silent=True, subsample=1.0,
        subsample_for_bin=320000, subsample_freq=0)

In [12]:
%time model2.fit(np.concatenate((np.array(train_hdf1['features'][:, features_2]), np.array(train_hdf2['features'][:, features_2]))), np.concatenate((np.array(train_hdf1['labels']), np.array(train_hdf2['labels']))), sample_weight=abs_weights)

CPU times: user 40min 57s, sys: 31.6 s, total: 41min 28s
Wall time: 5min 16s


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8686735616702099,
        feature_fraction=0.8, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='auc', min_child_samples=5,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
        reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
        save_binary=True, silent=True, subsample=1.0,
        subsample_for_bin=320000, subsample_freq=0)

In [13]:
%time model3.fit(np.concatenate((np.array(train_hdf1['features'][:, features_3]), np.array(train_hdf2['features'][:, features_3]))), np.concatenate((np.array(train_hdf1['labels']), np.array(train_hdf2['labels']))), sample_weight=abs_weights)

CPU times: user 40min 54s, sys: 33.2 s, total: 41min 27s
Wall time: 5min 16s


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.8686735616702099,
        feature_fraction=0.8, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='auc', min_child_samples=5,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, num_leaves=205, objective='binary', random_state=42,
        reg_alpha=0.18989485009024593, reg_lambda=0.37919679987403715,
        save_binary=True, silent=True, subsample=1.0,
        subsample_for_bin=320000, subsample_freq=0)

Public test prediction

In [14]:
test_hdf = h5py.File('test.hdf', mode = 'r')
predictions = (model_pion.predict_proba(np.array(test_hdf['feature']))[:, 1] +
               model_proton.predict_proba(np.array(test_hdf['feature']))[:, 1] + 
               model0.predict_proba(np.array(test_hdf['feature'][:, features_0]))[:, 1] + 
               model1.predict_proba(np.array(test_hdf['feature'][:, features_1]))[:, 1] + 
               model2.predict_proba(np.array(test_hdf['feature'][:, features_2]))[:, 1] + 
               model3.predict_proba(np.array(test_hdf['feature'][:, features_3]))[:, 1]) / 6
test_hdf.close()

In [15]:
pd.DataFrame(data={"prediction": predictions}, index=range(726095)).to_csv(
    "private_result.csv", index_label="id")

Private test prediction

In [19]:
test_hdf = h5py.File('test_private.hdf', mode = 'r')
predictions = (model_pion.predict_proba(np.array(test_hdf['features']))[:, 1] +
               model_proton.predict_proba(np.array(test_hdf['features']))[:, 1] + 
               model0.predict_proba(np.array(test_hdf['features'][:, features_0]))[:, 1] + 
               model1.predict_proba(np.array(test_hdf['features'][:, features_1]))[:, 1] + 
               model2.predict_proba(np.array(test_hdf['features'][:, features_2]))[:, 1] + 
               model3.predict_proba(np.array(test_hdf['features'][:, features_3]))[:, 1]) / 6
test_hdf.close()

In [28]:
pd.DataFrame(data={"prediction": predictions.astype('float32')}, index=range(1452188)).to_csv(
    "private_result.csv", index_label="id")